<a target="_blank" href="https://colab.research.google.com/github/OnurKerimoglu/water_bodies/blob/main/nb/satellite-water-bodies-keras-sm.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:

%matplotlib inline

In [ ]:
import os
import subprocess
import sys
import tensorflow as tf
if tf.test.is_gpu_available():
    HEIGHT, WIDTH = (512, 512)
    BATCH_SIZE = 16
    ENCODER = 'resnet50'
    DEVICE = 'cuda'
    FLAG_CUDA = True
    subprocess.check_call([sys.executable, "-m", "pip", "install", "segmentation-models"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "opencv-python"])
    # subprocess.check_call([sys.executable, "-m", "pip", "install", "albumentations"])
    percentdata2use=100
else:
    HEIGHT, WIDTH = (256, 256)
    BATCH_SIZE = 128
    ENCODER = 'resnet18'
    DEVICE = 'cpu'
    FLAG_CUDA = False
    percentdata2use=25
WEIGHTS = 'imagenet'
ROOTPATH = os.path.dirname(os.path.abspath(os.path.curdir))
print(f'rootpath: {ROOTPATH}, cuda available: {FLAG_CUDA}')

In [ ]:
import glob
import os

import cv2
import numpy as np
from matplotlib import pyplot as plt
os.environ["SM_FRAMEWORK"] = "tf.keras" 
import tensorflow as tf
import segmentation_models as sm
from sklearn.model_selection import train_test_split

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
franciscoescobar_satellite_images_of_water_bodies_path = kagglehub.dataset_download('franciscoescobar/satellite-images-of-water-bodies')

print('Data source import complete.')

In [ ]:
franciscoescobar_satellite_images_of_water_bodies_path

In [ ]:
Images_dir = os.path.join(franciscoescobar_satellite_images_of_water_bodies_path, 'Water Bodies Dataset', 'Images')
Masks_dir = os.path.join(franciscoescobar_satellite_images_of_water_bodies_path, 'Water Bodies Dataset', 'Masks')
image_paths = sorted(glob.glob(f'{Images_dir}/*'))
mask_paths = sorted(glob.glob(f'{Masks_dir}/*'))

num_images_to_use = round(len(image_paths) * percentdata2use / 100)
image_paths = image_paths[:num_images_to_use]
mask_paths = mask_paths[:num_images_to_use]

In [ ]:
len(mask_paths)

In [ ]:
#Capture training image info as a list
img_np_pr = []

for img_path in image_paths:
    #print(img_path)
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
    img = cv2.resize(img, (WIDTH, HEIGHT))
    # img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img = img/255.0
    img_np_pr.append(img)
    #train_labels.append(label)
#Convert list to array for machine learning processing        
img_np_pr = np.array(img_np_pr)

#Capture mask/label info as a list
mask_np_pr = [] 
for mask_path in mask_paths:
    mask = cv2.imread(mask_path, 0)       
    mask = cv2.resize(mask, (WIDTH, HEIGHT))
    #mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)
    mask = mask/255.0
    mask_np_pr.append(mask)
    #train_labels.append(label)
#Convert list to array for machine learning processing          
mask_np_pr = np.array(mask_np_pr)
mask_np_pr = np.expand_dims(mask_np_pr, axis=3)

In [ ]:
img_np_pr.shape

In [ ]:
mask_np_pr.shape

In [ ]:
f, axarr = plt.subplots(1,2)
num=2
axarr[0].imshow(img_np_pr[num])
axarr[1].imshow(mask_np_pr[num])

In [ ]:
img_np_pr[0]

In [ ]:
mask_np_pr[0]

In [ ]:
#Use customary x_train and y_train variables
X = img_np_pr
Y = mask_np_pr

x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# preprocess input
preprocess_input = sm.get_preprocessing(ENCODER)
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)

In [ ]:
# define model
model = sm.Unet(ENCODER, encoder_weights=WEIGHTS)
# bce_jaccard_loss = sm.losses.bce_jaccard_loss
dice_loss = sm.losses.DiceLoss()
bce_loss = sm.losses.BinaryCELoss()
# focal_loss = sm.losses.BinaryFocalLoss()
# total_loss = sm.losses.bce_dice_loss + (1 * focal_loss)
total_loss = bce_loss + dice_loss

model.compile(
    'Adam',
    # loss=bce_jaccard_loss,
    loss = total_loss,
    metrics=[sm.metrics.iou_score],)

# print(model.summary())

In [ ]:
# y_train_f = tf.cast(y_train, dtype=tf.float32)
# y_val_f = tf.cast(y_val, dtype=tf.float32)

In [ ]:
BATCH_SIZE

In [ ]:
x_train.shape

In [ ]:
x_val.shape

In [ ]:
y_val.shape

In [ ]:
f, axarr = plt.subplots(1,2)
num=3
axarr[0].imshow(x_val[num])
axarr[1].imshow(y_val[num])

In [ ]:
path_training_logs = os.path.join(ROOTPATH, 'training_logs')
if not os.path.exists(path_training_logs):
    os.makedirs(path_training_logs)

path_models = os.path.join(ROOTPATH, 'models', 'keras_tensorflow_unet', ENCODER)
if not os.path.exists(path_models):
    os.makedirs(path_models)
    
import keras
EScallback = keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=3)

TBcallback = keras.callbacks.TensorBoard(
    log_dir=path_models,
    histogram_freq=0,
    write_graph=True,
    write_images=False
)

CPcallback = keras.callbacks.ModelCheckpoint(
    os.path.join(path_models, 'best_model.weights.h5'), 
    save_weights_only=True, 
    save_best_only=True,
    mode='min')

RLRcallback =keras.callbacks.ReduceLROnPlateau()

In [ ]:
history = model.fit(
   x=x_train,
   y=y_train,
   batch_size=BATCH_SIZE,
   epochs=5,
   validation_data=(x_val, y_val),
   callbacks=[TBcallback, EScallback, CPcallback,RLRcallback],
   verbose=1
)


In [ ]:
accuracy = model.evaluate(x_val, y_val)

In [ ]:
accuracy

In [ ]:
print(history.history.keys())

In [ ]:

# summarize history for accuracy
plt.plot(history.history['iou_score'])
plt.plot(history.history['val_iou_score'])
plt.title('model iou score')
plt.ylabel('iou_score')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
fpath = '../models/water_bodies_EncResnet34_10epochs.keras'
model.save(fpath)

In [ ]:
from tensorflow import keras
model = keras.models.load_model(fpath, compile=False)

In [ ]:
#Test on a different image
#READ EXTERNAL IMAGE...
num = 0
image = cv2.imread(
    image_paths[num], 
    cv2.IMREAD_COLOR)     
image = cv2.resize(image, (SIZE_Y, SIZE_X))
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

image = preprocess_input(image)

In [ ]:
mask = cv2.imread(
    mask_paths[num], 
    0)     
mask = cv2.resize(mask, (SIZE_Y, SIZE_X))
# mask = np.expand_dims(mask, axis=0)

pred_mask = model.predict(
    np.expand_dims(image, axis=0)
    )
logits_mask = pred_mask.squeeze()
# logits_mask = cv2.cvtColor(pred_mask, cv2.COLOR_RGB2BGR)
pred_mask=tf.sigmoid(logits_mask)
pred_mask=(pred_mask > 0.5)*1.0

In [ ]:
pred_mask = model.predict(
    np.expand_dims(image, axis=0)
    )
pred_mask = pred_mask.squeeze()
pred_mask = tf.sigmoid(pred_mask)
pred_mask

In [ ]:
pred_mask

In [ ]:
f, axarr = plt.subplots(1,3)
axidx = 0
axarr[axidx].imshow(image)
axarr[axidx].set_title('Image')
axidx += 1
axarr[axidx].imshow(np.squeeze(mask), cmap='gray')
axarr[axidx].set_title('Mask')
axidx += 1
axarr[axidx].imshow(pred_mask)
axarr[axidx].set_title('Predicted Mask')